In [1]:
import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from sklearn import feature_extraction

In [2]:

from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
dataset = pd.read_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/IMDB Dataset.csv')
dataset.info()
dataset.head(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
one_hot = pd.get_dummies(dataset['sentiment'])
dataset = dataset.drop('sentiment', axis=1) 
dataset = dataset.join(one_hot)
dataset.head(5)

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


In [ ]:
def pre_processing(text):
  words=word_tokenize(text)
  words_wo_punc=[]
  for w in words:
    if w.isalpha():
      words_wo_punc.append(w.lower())
  clean_words=[]
  stop_words = set(stopwords.words("english"))
  for w in words_wo_punc:
    if w not in stop_words:
      clean_words.append(w)
  lemma=WordNetLemmatizer()
  lemma_words=[]
  for w in clean_words:
    lemma_words.append(lemma.lemmatize(w))
  return ' '.join(lemma_words)


In [ ]:
dataset['review'] = dataset['review'].apply(pre_processing)
dataset.to_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/pre_processed_data.csv', index=False)
dataset.head(5)

In [4]:
 dataset = pd.read_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/pre_processed_data.csv')
 dataset.head(5)

,review,negative,positive
0,one reviewer mentioned watching oz episode hoo...,0,1
1,wonderful little production br br filming tech...,0,1
2,thought wonderful way spend time hot summer we...,0,1
3,basically family little boy jake think zombie ...,1,0
4,petter mattei love time money visually stunnin...,0,1


In [5]:
from sklearn.model_selection import train_test_split
X =dataset.iloc[:, 0].values
Y = dataset.iloc[:, 2].values


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 10000
max_len = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X= pad_sequences(sequences, maxlen=max_len)
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=60, test_size=0.20)
print("Number of train samples: ", len(X_train))
print("Number of test samples: ", len(X_test))

Number of train samples:  40000
Number of test samples:  10000


In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_gru.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

from keras.layers import Embedding
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
model1 = Sequential()
model1.add(layers.Embedding(max_words, 64))
model1.add(layers.GRU(10,dropout=0.3))
model1.add(layers.Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, batch_size=128, epochs=500, validation_data=(X_test, y_test),verbose=1,callbacks = [model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_gru.h5')
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8835999965667725
